## Calculating a Typical Meteorological Year

This notebook walks through the process of calculating a [Typical Meteorological Year](https://nsrdb.nrel.gov/data-sets/tmy), an hourly dataset used for applications in energy and building systems modeling. Because this represents average rather than extreme conditions, an TMY dataset is not suited for designing systems to meet the worst-case conditions occurring at a location. 

The TMY methodology here mirrors that of the Sandia/NREL TMY3 methodology, and uses historic and projected downscaled climate data available through the Cal-Adapt: Analytics Engine catalog. As this methodology heavily weights the solar radiation input data, be aware that the final selection of "typical" months may not be typical for other variables. 

The Analytics Engine at present has an *Average Meteorological Year* functionality. The methods shown throughout this notebook will soon replace the underlying backend `climakitae` code for the AMY in order to better address our user needs, i.e., we are working to replace the AMY with the TMY methods. We provide this walkthrough to demonstrate confidence in the "AMY to TMY" conversion process for our users in the meantime. 

<span style="color:#FF0000">**Note**: <span style="color:#000000"> At present, one of the core variables (direct normal irradiance) required to calculate a TMY is not currently available in the Cal-Adapt: Analytics Engine catalog but is coming soon. Until DNI is available, the resulting TMY files produced in this notebook may not accurately reflect typical conditions. This notebook will be updated once DNI is available. 

### Step 0: Set-up

Import the [climakitae](https://github.com/cal-adapt/climakitae) library and other dependencies.

In [ ]:
import climakitae as ck
import pandas as pd
import xarray as xr
import numpy as np
import pkg_resources
from tqdm.auto import tqdm  # Progress bar

import warnings 
warnings.filterwarnings("ignore")

from climakitae.data_export import write_tmy_file

To use climakitae, load a new application:

In [ ]:
app = ck.Application()

### Step 1: Grab and process all required input data

The [TMY3 method](https://www.nrel.gov/docs/fy08osti/43156.pdf) selects a "typical" month based on ten daily variables: max, min, and mean air and dew point temperatures, max and mean wind speed, global irradiance and direct irradiance. Some of these variables are already available in the Analytics Engine data catalog, and some we will need to calculate ourselves.  

#### Step 1a: Select location of interest
TMYs are usually calculated for a specific location of interest, like a building or power plant. Here, we will use a known weather station location, via their latitude and longitude to extract the data that we need to calculate the TMY.  In the example below, we will look specifically at Los Angeles International Airport, but will note in the code below how you can provide your own location coordinates too. 

In [ ]:
# read in station file of CA HadISD stations
stn_file = pkg_resources.resource_filename("climakitae", "data/hadisd_stations.csv")
stn_file = pd.read_csv(stn_file, index_col=[0])
stn_file.head(5)

In [ ]:
# grab airport
station_name = "Los Angeles International Airport"
one_station = stn_file.loc[stn_file['station'] == station_name]
stn_lat = one_station.LAT_Y.item()
stn_lon = one_station.LON_X.item()
stn_lat, stn_lon

Alternatively, you may want to provide your own location instead of one of the HadISD stations above. If so, uncomment the cell below by removing the `#` symbol.

In [ ]:
## provide your own location, via latitude and longitude coordinates
# stn_lat = YOUR_LAT_HERE
# stn_lon = YOUR_LON_HERE
# station_name = 'YOUR_STATION_NAME_HERE'

#### Step 1b: Variables in catalog
When selecting data, there are several considerations to be aware of. The recommended minimum input of data is 15-20 years worth of daily data. First, we will pre-load some data options to ensure that the same constraints are applied to every variable we retrieve from the catalog and calculate. For example, we will process the data for our designated station location (latitude, and longitude) at 45 km over the 1990-2020 period. For data post-2014, we will utilize SSP 3-7.0, although scenario selection in the near-future is relatively independent. If calculating a TMY for the far-future, **carefully consider which scenario SSP to include**, as there will be **significant** differences present. 

In [ ]:
# default selections applicable to all variables selected
app.selections.data_type = "Gridded"
app.selections.area_average = "Yes"
app.selections.scenario_historical = ["Historical Climate"]
app.selections.scenario_ssp = ["SSP 3-7.0 -- Business as Usual"] # Important based on time period considered!!
app.selections.time_slice = (1990, 2020)
app.selections.timescale = "daily"
app.selections.resolution = "45 km"
app.selections.cached_area = ['coordinate selection']
app.selections.latitude = (stn_lat-0.2, stn_lat+0.2)
app.selections.longitude = (stn_lon-0.2, stn_lon+0.2)

Now that we have set up default settings, let's start retrieving data! First, retrieve min, max, and mean air temperature: 

In [ ]:
# max air temp
app.selections.variable = "Maximum air temperature at 2m"
app.selections.units = "degC"
max_airtemp_data = app.retrieve()

# min air temp
app.selections.variable = "Minimum air temperature at 2m"
app.selections.units = "degC"
min_airtemp_data = app.retrieve()

# mean air temperature
app.selections.variable = "Air Temperature at 2m" 
app.selections.units = "degC"
mean_airtemp_data = app.retrieve()
mean_airtemp_data.name = "Mean air temperature at 2m" # rename for clarity

Retrieve max and mean wind speed:

In [ ]:
# max wind speed
app.selections.variable = "Maximum wind speed at 10m"
app.selections.units = "m s-1"
max_windspd_data = app.retrieve()

# mean wind speed
app.selections.variable = "Mean wind speed at 10m"
app.selections.units = "m s-1"
mean_windspd_data = app.retrieve()

#### Step 1c: Variables that need to be calculated
Next, we will need to retrieve **hourly** data from the catalog to calculate the following variables, as they are not natively within the Analytics Engine data catalog. 
- Max, min, and mean dew point temperature
- Global horizontal irradiance
- Direct normal irradiance **-- coming to the AE catalog soon**

In [ ]:
# switch to hourly timescale
app.selections.timescale = "hourly"

Next, calculate max, min, and mean dew point temperature:

In [ ]:
# dew point temperature
app.selections.variable = "Dew point temperature"
app.selections.units = "degC"
dewpt_data = app.retrieve()

In [ ]:
max_dewpt_data = dewpt_data.resample(time="1D").max() # daily max dewpoint temp
max_dewpt_data.name = "Daily max dewpoint temperature" # rename for clarity

min_dewpt_data = dewpt_data.resample(time="1D").min() # daily min dewpoint temp
min_dewpt_data.name = "Daily min dewpoint temperature" # rename for clarity

mean_dewpt_data = dewpt_data.resample(time="1D").mean() # daily mean dewpoint temp
mean_dewpt_data.name = "Daily mean dewpoint temperature" # rename for clarity

Next, retrieve global horizontal irradiance. GHI is within the Analytics Enginge catalog at daily resolutions, but for the TMY methodology, we need to calculate the total accumulated GHI received over the course of the day, so we will retrieve hourly data instead and calculate the necessary information below. The same process is repeated for the direct normal irradiance (once the data is in the AE catalog). 

In [ ]:
# global irradiance
app.selections.variable = "Instantaneous downwelling shortwave flux at bottom"
global_irradiance_data = app.retrieve()
global_irradiance_data.name = "Global horizontal irradiance" # rename for clarity
total_ghi_data = global_irradiance_data.resample(time="1D").sum() # total global horizontal irradiance (accumulation of hourly data over a 24-hour period)

In [ ]:
## ONCE IN CATALOG

# # direct normal irradiance
# app.selections.variable = "VARIABLE_NAME_HERE"
# direct_irradiance_data = app.retrieve()
# direct_irradiance_data.name = "Direct normal irradiance" # rename for clarity
# total_dni_data = direct_irradiance_data.resample(time="1D").sum() # total direct normal irradiance (accumulation of hourly data over a 24-hour period)

#### Step 1d: Load all variables
Now that we have all of our data retrieved and calculated, it is time to actually load the data into memory. Previously, xarray has lazily loaded the data, but we will actually grab it now. 

In [ ]:
all_vars = xr.merge([max_airtemp_data.squeeze(), min_airtemp_data.squeeze(), mean_airtemp_data.squeeze(),
                     max_dewpt_data.squeeze(), min_dewpt_data.squeeze(), mean_dewpt_data.squeeze(),
                     max_windspd_data.squeeze(), mean_windspd_data.squeeze(),
                     total_ghi_data.squeeze()]) #, total_dni_data.squeeze()]) # uncomment once DNI in catalog

In [ ]:
# load all indices in
all_vars = all_vars.compute()

In [ ]:
all_vars

### Step 2: Calculate cumultative distribution functions

For the TMY, the cumulative distribution function gives the proportion of values that are less than or equal to a specified value of the index. In this case, we want to identify months that are as close to the long-term climatology for each variable as possible, indicating months that are "typical".  

#### Step 2a: Calculate long-term climatology CDFs for each index
First, we need to calculate the long-term climatology for each index for each month so we can establish the baseline pattern. 

In [ ]:
def compute_cdf(da): 
    """Compute the cumulative density function for an input DataArray"""
    da_np = da.values # Get numpy array of values
    num_samples = 1024 # Number of samples to generate
    count, bins_count = np.histogram( # Create a numpy histogram of the values 
        da_np, bins = np.linspace(
            da_np.min(), # Start at the minimum value of the array 
            da_np.max(), # End at the maximum value of the array 
            num_samples
        )
    )
    cdf_np = np.cumsum(count/sum(count)) # Compute the CDF 
    
    # Turn the CDF array into xarray DataArray 
    # New dimension is the bin values 
    cdf_da = xr.DataArray( 
        [bins_count[1:],cdf_np],
        dims=["data","bin_number"],
        coords = {
            "data":["bins","probability"],
        }
    )
    cdf_da.name = da.name
    return cdf_da

def get_cdf_by_sim(da): 
    # Group the DataArray by simulation
    return da.groupby("simulation").apply(compute_cdf)

def get_cdf_by_mon_and_sim(da): 
    # Group the DataArray by month in the year 
    return da.groupby("time.month").apply(get_cdf_by_sim)

def get_cdf(ds): 
    """Get the cumulative density function. 
    
    Parameters
    -----------
    ds: xr.Dataset
        Input data to compute CDF for
    Returns
    -------
    xr.Dataset
    """
    return ds.apply(get_cdf_by_mon_and_sim)

In [ ]:
cdf_climatology = get_cdf(all_vars)

Next, we set up a handy plotting function so that we can view the CDF patterns for each variable. 

In [ ]:
def plot_one_var_cdf(cdf_da): 
    """Plot CDF for a single variable 
    Written to function for the unique configuration of the CDF DataArray object
    Silences an annoying hvplot warning 
    Will show every simulation together on the plot 
    
    Parameters
    -----------
    cdf: xr.DataArray 
       Cumulative density function for a single variable 
    
    Returns 
    -------
    panel.layout.base.Column
        Hvplot lineplot 
        
    """
    prob_da = cdf_da.sel(data="probability",drop=True).rename("probability") # Grab only probability da 
    bins_da = cdf_da.sel(data="bins",drop=True).rename("bins") # Grab just bin values
    ds = xr.merge([prob_da,bins_da]) # Merge the two to form a single Dataset object
    cdf_pl = ds.hvplot(
        "bins","probability", 
        by="simulation", # Simulations should all be displayed together 
        widget_location="bottom",
        grid=True,
        xlabel="{0} ({1})".format(var,cdf_da.attrs["units"]), 
        xlim=(bins_da.min().item(), bins_da.max().item()), # Fix the x-limits for all months
        ylabel="Probability (0-1)", 
    )
    return cdf_pl 

In the plot below, we'll display maximum air temperature to assess the climatological CDF pattern, but you can modify the variable here to one of your choosing to see the pattern too! Also select a different month by moving the slider bar to see the pattern throughout the year. 

In [ ]:
# Choose your desired variable
var = "Maximum air temperature at 2m"

# Make the plot
cdf_plot = plot_one_var_cdf(cdf_climatology[var])
display(cdf_plot)

#### Step 2b: Calculate CDFs for each index for all months

Next, we will calculate CDF for each month and each variable, for which we ultimatley will compare against climatology. For the individual months, we must also exclude the period of time during a major volcanic eruption (Pinatubo: June 1991 to December 1994) as the aerosols have an impact on solar variables. The cells below functions exlude this data from our data next. 

In [ ]:
def get_cdf_monthly(ds):
    """Get the cumulative density function by unique mon-yr combos
    
    Parameters
    -----------
    ds: xr.Dataset
        Input data to compute CDF for
    Returns
    -------
    xr.Dataset
    """
    def get_cdf_mon_yr(da): 
        return da.groupby("time.year").apply(get_cdf_by_mon_and_sim)
    return ds.apply(get_cdf_mon_yr)

In [ ]:
cdf_monthly = get_cdf_monthly(all_vars)

Now we'll remove the volcanic years: 

In [ ]:
# Remove the years for the Pinatubo eruption 
cdf_monthly = cdf_monthly.where(
    (~cdf_monthly.year.isin([1991,1992,1993,1994])), 
    np.nan, drop=True)

Like the climatology CDF figure above, let's check out the individual months next. You can modify the variable, and month-year to display too. 

In [ ]:
# Choose your desired variable
var = "Maximum air temperature at 2m"

# Make the plot
cdf_plot_mon_yr = plot_one_var_cdf(cdf_monthly[var])
display(cdf_plot_mon_yr)

### Step 3: Compare climatology CDF to monthly CDF for each variable

Now that we hvae the distributions for the long-term climatology of our 30-year period, and the corresponding distribution for each month in that 30-year period, we need to assess how each individual month compares to the long-term climatology. In essence, we are looking for the individual months that best capture the climatology distribution. 

#### Step 3a: Calculate the Finkelstein-Schafer statistic 
The [Finkelstein-Schafer statistic](https://academic.oup.com/biomet/article-abstract/58/3/641/233677) determines the absolute difference between the long-term climatology and candidate CDF profiles, and considers the number of days within each month. We will use a handy function `fs_statistic` to calculate this below. 

In [ ]:
def fs_statistic(cdf_climatology, cdf_month):
    """
    Calculates the Finkelstein-Schafer statistic:
    Absolute difference between long-term climatology and candidate CDF, divided by number of days in month
    """
    days_per_mon = xr.DataArray(
        data=[31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31], 
        coords={"month":np.arange(1,13)}
    )
    fs_stat = abs(cdf_monthly - cdf_climatology).sel(data="probability") / days_per_mon
    return fs_stat

In [ ]:
all_vars_fs = fs_statistic(cdf_climatology, cdf_monthly)

#### Step 3b: Weight the F-S statistic

Next, we weight the F-S statistic results based on the input variables. The [TMY3](https://www.nrel.gov/docs/fy08osti/43156.pdf) method places a higher weight to the solar variables (global irradiance and direct irradiance), which we follow here. 

In [ ]:
def compute_weighted_fs(da_fs):
    """Weights the F-S statistics based on TMY3 methodology"""
    weights_per_var = {
        'Maximum air temperature at 2m':1/20,
        'Minimum air temperature at 2m':1/20,
        'Mean air temperature at 2m':2/20,
        'Daily max dewpoint temperature':1/20,
        'Daily min dewpoint temperature':1/20,
        'Daily mean dewpoint temperature':2/20,
        'Maximum wind speed at 10m':1/20,
        'Mean wind speed at 10m':1/20,
        'Global horizontal irradiance':5/20
        # 'Direct normal irradiance':5/20 ## uncomment once in catalog
    }

    for var, weight in weights_per_var.items(): 
        # Multiply each variable by it's appropriate weight 
        da_fs[var] = da_fs[var]*weight 
    return da_fs

In [ ]:
weighted_fs = compute_weighted_fs(all_vars_fs)

#### Step 3c: Select candidate months for consideration
Once weighted, we select the top candidate months for each month that have lowest weighted sums, meaning that these candidate months are the closest to the long-term climatology for that month. 

In [ ]:
# Sum 
weighted_fs_sum = weighted_fs.to_array().sum(dim=["variable","bin_number"]).drop(["data"])

# Pass the weighted F-S sum data for simplicity
ds = weighted_fs_sum

df_list = []
num_values = 1 # Selecting the top value for now, persistence statistics calls for top 5
for sim in ds.simulation.values: 
    for mon in ds.month.values:
        da_i = ds.sel(month=mon, simulation=sim)
        top_xr = da_i.sortby(da_i, ascending=True)[:num_values].expand_dims(["month","simulation"])
        top_df_i = top_xr.to_dataframe(name="top_values")
        df_list.append(top_df_i)
        
# Concatenate list together for all months and simulations
top_df = pd.concat(df_list).drop(columns=["top_values"]).reset_index()
top_df

The data table above represents the ideal months that represent the long term climatology based on the 10 indices for the 4 simulations in the Analytics Engine catalog. Meaning, for a "typical" meteorological year, CESM2 data for Jan will come from Jan 2003, data for Feb will come from 1996, and so on. In the next step, we will generate the resulting "TMY" file that is commonly used in such applications. 

### Step 4: Generate the TMY data outputs

Generally, the following data is outputted using the TMY months:
- Date & time (UTC)
- Air temperate at 2m [°C]
- Dew point temperature [°C]
- Relative humidity [%]
- Global horizontal irradiance [W/m2]
- Direct normal irradiance [W/m2] -- ***coming soon to AE***
- Diffuse horizontal irradiance [W/m2]
- Downwelling infrared radiation [W/m2]
- Wind speed at 10m [m/s]
- Wind direction at 10m [°]
- Surface air pressure [Pa]

The following function will retrieve all of this data for the designated TMY month and concatenate it together into a pandas dataframe so that we can export it as a csv file. We'll do this next. 

In [ ]:
def generate_tmy_data(top_df): 
    """Generate typical meteorological year data 
    Ouput will be a list of dataframes per simulation. 
    Print statements throughout the function indicate to the user the progress of the computation. 
    
    Parameters
    -----------
    top_df: pd.DataFrame 
        Table with column values month, simulation, and year 
        Each month-sim-yr combo represents the top candidate that has the lowest weighted sum from the FS statistic 
    
    Returns
    --------
    dict of str:pd.DataFrame 
        Dictionary in the format of {simulation:TMY corresponding to that simulation} 
    
    """
    ## ================== GET DATA FROM CATALOG ==================
    vars_and_units = {
        'Air Temperature at 2m':'degC',
        'Dew point temperature':'degC',
        'Relative humidity':'[0 to 100]', 
        'Instantaneous downwelling shortwave flux at bottom':'W m-2',
        # 'direct irradiance here':'W m-2', ## uncomment once in catalog 
        'Shortwave surface downward diffuse irradiance':'W m-2',
        'Instantaneous downwelling longwave flux at bottom':'W m-2',
        'Wind speed at 10m':'m s-1',
        'Wind direction at 10m':'degrees',
        'Surface Pressure':'Pa' # Pa
    }
    
    # Set up shared catalog access settings
    app.selections.data_type = "Gridded"
    app.selections.area_average = "No"
    app.selections.scenario_historical = ["Historical Climate"]
    app.selections.scenario_ssp = ["SSP 3-7.0 -- Business as Usual"] # Important based on time period considered!!
    app.selections.timescale = "hourly"
    app.selections.resolution = "45 km"
    app.selections.cached_area = ['coordinate selection']
    app.selections.latitude = (stn_lat-0.2, stn_lat+0.2)
    app.selections.longitude = (stn_lon-0.2, stn_lon+0.2)
    app.selections.time_slice = (1990, 2020)

    # Loop through each variable and grab data from catalog 
    all_vars_list = []
    print("STEP 1: RETRIEVING HOURLY DATA FROM CATALOG\n")
    for var, units in vars_and_units.items(): 
        print("Retrieving data for {0}".format(var), end="... ")
        app.selections.variable = var # Set variable
        app.selections.units = units # Set units  
        data_by_var = app.retrieve() # Retrieve data from catalog 

        # Drop unwanted coords
        data_by_var = data_by_var.squeeze().drop(['lakemask','landmask','x','y','Lambert_Conformal'])

        all_vars_list.append(data_by_var) # Append to list 
        print("complete!")

    # Merge data from all variables into a single xr.Dataset object 
    all_vars_ds = xr.merge(all_vars_list)


    ## ================== CONSTRUCT TMY ==================
    print("\nSTEP 2: CALCULATING TYPICAL METEOROLOGICAL YEAR PER MODEL SIMULATION\nProgress bar shows code looping through each month in the year.\n")
    tmy_df_all = {}
    for sim in all_vars_ds.simulation.values: 
        df_list = []
        print("Calculating TMY for simulation: {0}".format(sim))
        for mon in tqdm(np.arange(1,13,1)): 
            # Get year corresponding to month and simulation combo 
            year = top_df.loc[(top_df["month"] == mon) & (top_df["simulation"] == sim)].year.item()

            # Select data for unique month, year, and simulation
            data_at_stn_mon_sim_yr = all_vars_ds.sel(simulation=sim, time="{0}-{1}".format(mon, year)).expand_dims("simulation")

            # Reformat as dataframe 
            df_by_mon_sim_yr = data_at_stn_mon_sim_yr.to_dataframe()
            df_by_mon_sim_yr = df_by_mon_sim_yr.reset_index()

            # Reformat time index to remove year
            df_by_mon_sim_yr["time"] = pd.to_datetime(df_by_mon_sim_yr["time"].values).strftime("%m-%d %H:%M:%S")
            df_list.append(df_by_mon_sim_yr)

        # Concatenate all DataFrames together 
        tmy_df_by_sim = pd.concat(df_list)
        tmy_df_all[sim] = tmy_df_by_sim
        
    return tmy_df_all # Return dict of TMY by simulation 

In the next cell, we will run the `generate_tmy_data` function which will retrieve, subset, and format the data for each month according to the TMY months for all requested variables. We have included print statements so you can watch the progress for each variable in each month as it builds. 

<span style="color:#FF0000">**Note**: <span style="color:#000000"> This will take time! On the Analytics Engine JupyterHub, this takes approximately 20 minutes. Progress bars will indicate the status of generating the TMY data for each simulation. 

In [ ]:
tmy_data_to_export = generate_tmy_data(top_df)

Let's observe what the TMY data looks like for one of the simulations:

In [ ]:
simulation = "WRF_CESM2_r11i1p1f1"
tmy_data_to_export[simulation]

Next, we visualize the TMY data itself.

In [ ]:
tmy_data_to_export[simulation].plot(x='time',y=['Air Temperature at 2m',
                                                'Dew point temperature',
                                                'Relative humidity',
                                                'Instantaneous downwelling shortwave flux at bottom',
                                                'Shortwave surface downward diffuse irradiance',
                                                'Instantaneous downwelling longwave flux at bottom',
                                                'Wind speed at 10m', 'Wind direction at 10m', 'Surface Pressure'], 
                                    title='Typical Meteorological Year ({})'.format(simulation),
                                    subplots=True, figsize=(10,8), legend=True);

Lastly, let's export the TMY data below as csv files. There will be a file per simulation downloaded. When utilizing TMY data in your own workflows, we recommend that **all simulations are considered** in your analyses, especially for future scenarios. 

In [ ]:
for sim, tmy in tmy_data_to_export.items(): 
    filename = 'TMY_{0}_{1}'.format(station_name.replace(" ", "_"), sim).lower()
    write_tmy_file(filename, tmy_data_to_export[sim], location_name=station_name, file_ext="epw")